In [2]:
import os

In [3]:
os.chdir("../")



In [4]:
%pwd


'd:\\Kidney-Disease-Classification-Deep-Learning-Project'

In [5]:
#https://dagshub.com/Harishrsharma/Kidney-Disease-Classification-Deep-Learning-Project.mlflow

In [6]:
import mlflow
mlflow.set_tracking_uri("https://dagshub.com/Harishrsharma/Kidney-Disease-Classification-Deep-Learning-Project.mlflow")

import dagshub
dagshub.init(repo_owner='Harishrsharma', repo_name='Kidney-Disease-Classification-Deep-Learning-Project', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

Accessing as Harishrsharma

Initialized MLflow to track repo "Harishrsharma/Kidney-Disease-Classification-Deep-Learning-Project"

Repository Harishrsharma/Kidney-Disease-Classification-Deep-Learning-Project initialized!

In [ ]:
# import dagshub
# import mlflow
# mlflow.set_tracking_uri("https://dagshub.com/Harishrsharma/Kidney-Disease-Classification-Deep-Learning-Project.mlflow")


# # Initialize DagsHub integration first
# dagshub.init(
#     repo_owner="Harishrsharma",
#     repo_name="Kidney-Disease-Classification-Deep-Learning-Project",
#     mlflow=True
# )

# # (Optional) verify the URI
# print("Tracking URI:", mlflow.get_tracking_uri())

# # Test MLflow logging
# with mlflow.start_run():
#     mlflow.log_param("parameter name", "value")
#     mlflow.log_metric("metric name", 1)


Accessing as Harishrsharma

Initialized MLflow to track repo "Harishrsharma/Kidney-Disease-Classification-Deep-Learning-Project"

Repository Harishrsharma/Kidney-Disease-Classification-Deep-Learning-Project initialized!

Tracking URI: https://dagshub.com/Harishrsharma/Kidney-Disease-Classification-Deep-Learning-Project.mlflow
🏃 View run bouncy-shrimp-994 at: https://dagshub.com/Harishrsharma/Kidney-Disease-Classification-Deep-Learning-Project.mlflow/#/experiments/0/runs/b969f53ea98b47d68230ce745b08bfd0
🧪 View experiment at: https://dagshub.com/Harishrsharma/Kidney-Disease-Classification-Deep-Learning-Project.mlflow/#/experiments/0


In [7]:
import sys
print(sys.executable)

d:\Kidney-Disease-Classification-Deep-Learning-Project\myenv\Scripts\python.exe


In [8]:
import os
import sys

# Get the Current Working Directory (CWD) of the notebook process
current_dir = os.getcwd() 

# Assuming your project root is the CWD, you can define 'src_dir' relative to it
# If 'src' is directly in the CWD (project_root/src)
src_dir = os.path.join(current_dir, 'src') 

# If the notebook is deep inside the project and you need to go up levels:
# current_dir = os.getcwd() 
# project_root = os.path.abspath(os.path.join(current_dir, os.pardir, os.pardir)) # Example: go up two levels
# src_dir = os.path.join(project_root, 'src') 


# Add the 'src' directory to the system path
if src_dir not in sys.path:
    sys.path.insert(0, src_dir)



In [9]:
import tensorflow as tf

Accesing the Model which we have created in our artifact folder 

In [11]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [12]:
from dataclasses import dataclass
from pathlib import Path


#entity class config
@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [13]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [14]:
#Configuration Manager
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/Harishrsharma/Kidney-Disease-Classification-Deep-Learning-Project.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [15]:
pip show mlflow

Name: mlflowNote: you may need to restart the kernel to use updated packages.

Version: 2.2.2
Summary: MLflow: A Platform for ML Development and Productionization
Home-page: https://mlflow.org/
Author: Databricks
Author-email: 
License: Apache License 2.0
Location: d:\kidney-disease-classification-deep-learning-project\myenv\lib\site-packages
Requires: alembic, click, cloudpickle, databricks-cli, docker, entrypoints, Flask, gitpython, importlib-metadata, Jinja2, markdown, matplotlib, numpy, packaging, pandas, protobuf, pyarrow, pytz, pyyaml, querystring-parser, requests, scikit-learn, scipy, shap, sqlalchemy, sqlparse, waitress
Required-by: 


In [16]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [17]:
#component files
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")
    # def log_into_mlflow(self):
 
    # # Set the MLflow tracking URI (to your DagsHub tracking server)
    #     mlflow.set_tracking_uri(self.config.mlflow_uri)
    #     tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

    #     with mlflow.start_run():
    #         # Log parameters and metrics
    #         mlflow.log_params(self.config.all_params)
    #         mlflow.log_metrics(
    #             {"loss": self.score[0], "accuracy": self.score[1]}
    #         )

    #         # DagsHub does not support model registry API -> log as artifact only
    #         mlflow.keras.log_model(self.model, "model")

    #         print("✅ Model, params, and metrics logged successfully to MLflow/DagsHub.")

In [18]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2025-10-11 00:11:32,128: INFO: common: YAML file loaded successfully: D:\Kidney-Disease-Classification-Deep-Learning-Project\config\config.yaml]
[2025-10-11 00:11:32,137: INFO: common: YAML file loaded successfully: D:\Kidney-Disease-Classification-Deep-Learning-Project\params.yaml]
[2025-10-11 00:11:32,139: INFO: common: created directory at: artifacts]
Found 139 images belonging to 2 classes.
9/9 [==============================] - 27s 3s/step - loss: 1.3159 - accuracy: 0.5180
[2025-10-11 00:12:00,308: INFO: common: json file saved at: scores.json]


2025/10/11 00:12:01 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2025-10-11 00:12:05,051: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\91773\AppData\Local\Temp\tmpnv2k5gdm\model\data\model\assets
[2025-10-11 00:12:06,556: INFO: builder_impl: Assets written to: C:\Users\91773\AppData\Local\Temp\tmpnv2k5gdm\model\data\model\assets]


d:\Kidney-Disease-Classification-Deep-Learning-Project\myenv\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'VGG16Model'.
2025/10/11 00:15:56 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.
